# 언어모델의 편견 및 한계
앞에서 다뤘던 것 처럼 사전학습된 언어모델은 문제를 해결할때 강력한 도구라고 할 수 있습니다. 하지만 이러한 모델에도 한계는 있습니다. 사전학습을 위해 주로 인터넷상에서 수집했던 방대한 양의 텍스트 데이터에서는 좋은 컨텐츠도 있지만 좋지 못한 컨텐츠도 있기 때문입니다.

살짝 민감한 얘기일 수 있지만, 빈칸채우기(**fill-mask**) 문제를 BERT pipeline을 통해 실행해보면서 구체적인 예를 확인해보겠습니다.

In [3]:
!pip install -q transformers[sentencepiece]

     |████████████████████████████████| 3.3 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 42.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.5 MB/s 
     |████████████████████████████████| 895 kB 58.6 MB/s 
     |████████████████████████████████| 61 kB 383 kB/s 
     |████████████████████████████████| 1.2 MB 37.3 MB/s 


In [31]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].", top_k=4)
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].", top_k=4)
print([r["token_str"] for r in result])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman']
['nurse', 'maid', 'teacher', 'waitress']


위의 코드는 입력 문장 내에서 남성/여성의 직업에 대응되는 단어를 마스킹 한 뒤, 마스킹된 위치에 나올 단어를 예측합니다. 남성의 직업은 `목수, 변호사, 농부, 사업가`등의 단어가 예측되었지만 여성의 직업은 `간호사, 메이드, 교사, 웨이트리스`등의 단어가 예측되었습니다. 전체적으로 편향된 답변이 나온것을 확인 할 수 있습니다. BERT가 인터넷 전체에서 데이터를 모으지 않고 중립적인 데이터인 Wikipedia, BookCorpus에 대해서 학습했음에도 불구하고 위와 같은 편향이 존재합니다.   

그러므로 이러한 인공지능 모델을 사용할 때는 모델이 학습한 데이터에 따라 성차별, 인종차별, 혐오 발언등의 컨텐츠를 생성해 낼 수 있다는 점을 염두에 두어야 합니다. 